### Gradient Boosting pour la prédiction de la marge commerciale (sales_margin) avec colonnes éclatées



# %%
# Importation des bibliothèques nécessaires

In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lightgbm import LGBMRegressor
import numpy as np
import matplotlib.pyplot as plt

# %%
# Chargement des données

In [71]:

chemin = "C:/Users/zineb/OneDrive/Bureau/hetic/DA/silver/data_clean.csv"
df = pd.read_csv(chemin)



# %%
# Conversion des colonnes en type approprié

In [72]:


# 
# Préparation des données

# Gérer les valeurs manquantes
# Remplacer les virgules par des points si nécessaire
df['revenue'] = df['revenue'].str.replace(',', '.', regex=True)

# Convertir la colonne revenue en type float
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').astype(float)


# verification des colonnes engage_at et close_at sont au format datetime
df['engage_at'] = pd.to_datetime(df['engage_at'], errors='coerce')
df['close_at'] = pd.to_datetime(df['close_at'], errors='coerce')

# Remplir les valeurs de close_value avec 0 pour les états engaging, prospecting, et lost
df.loc[df['deal_stage'].isin(['engaging', 'prospecting', 'lost']), 'close_value'] = 0



# Calculer la marge commerciale (sales_margin)
df['sales_margin'] = df.apply(
    lambda row: 0 if row['close_value'] == 0 else row['close_value'] - row['sales_price'],
    axis=1
)


# Calcul de la durée du cycle de vente (sales_cycle_days)
df['sales_cycle_days'] = (df['close_at'] - df['engage_at']).dt.days



# Vérifier les valeurs manquantes dans sales_margin
print("Nombre de NaN dans sales_margin avant traitement :", df['sales_margin'].isnull().sum())

# Remplacer les NaN dans sales_margin par 0
df['sales_margin'] = df['sales_margin'].fillna(0)


# Vérifier à nouveau pour s'assurer qu'il n'y a plus de NaN
print("Nombre de NaN dans sales_margin après traitement :", df['sales_margin'].isnull().sum())


# Afficher les premières lignes pour vérifier
print(df[['close_value', 'sales_price', 'sales_margin', 'sales_cycle_days']].head())
df = df.dropna(subset=['sales_margin'])


# Créer des colonnes temporelles dérivées
df['engage_year'] = df['engage_at'].dt.year
df['engage_month'] = df['engage_at'].dt.month
df['engage_day'] = df['engage_at'].dt.day
df['engage_weekday'] = df['engage_at'].dt.weekday  # Lundi=0, Dimanche=6

df['close_year'] = df['close_at'].dt.year
df['close_month'] = df['close_at'].dt.month
df['close_day'] = df['close_at'].dt.day
df['close_weekday'] = df['close_at'].dt.weekday

# Calculer la durée entre "engage_at" et "close_at" si applicable
df['sales_cycle_days'] = (df['close_at'] - df['engage_at']).dt.days

# Remplir les valeurs manquantes

df['sales_cycle_days'] = df['sales_cycle_days'].fillna(0)







Nombre de NaN dans sales_margin avant traitement : 664
Nombre de NaN dans sales_margin après traitement : 0
   close_value  sales_price  sales_margin  sales_cycle_days
0          0.0          550           0.0               7.0
1          0.0          550           0.0              13.0
2          0.0          550           0.0              69.0
3          0.0          550           0.0               2.0
4          0.0          550           0.0               2.0


C:\Users\zineb\AppData\Local\Temp\ipykernel_7500\267395544.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['engage_at'] = pd.to_datetime(df['engage_at'], errors='coerce')
C:\Users\zineb\AppData\Local\Temp\ipykernel_7500\267395544.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['close_at'] = pd.to_datetime(df['close_at'], errors='coerce')


# %%
# Sélection des colonnes pertinentes

In [73]:
# 2. Supprimer les colonnes non pertinentes

columns_to_keep = [ 'sales_agent', 'product', 'account', 'deal_stage',
       'engage_at', 'close_at', 'close_value', 'sector', 'series',
       'sales_price', 'manager', 'office_location', 'regional_office',
       'year_founded_at', 'revenue', 'number_of_employees', 'sales_margin',
       'sales_cycle_days', 'engage_year', 'engage_month', 'engage_day',
       'engage_weekday', 'close_year', 'close_month', 'close_day',
       'close_weekday']

# Sélection des colonnes pertinentes
df = df[columns_to_keep]
df  = df .drop(columns=[ 'engage_at', 'close_at', 'close_value', 'year_founded_at', 'engage_year', 'close_year', 'close_month', 'close_day',
       'close_weekday', 'deal_stage',  'sales_cycle_days'])

# Vérification
print("Colonnes conservées :", df.columns.tolist())


Colonnes conservées : ['sales_agent', 'product', 'account', 'sector', 'series', 'sales_price', 'manager', 'office_location', 'regional_office', 'revenue', 'number_of_employees', 'sales_margin', 'engage_month', 'engage_day', 'engage_weekday']


# %%
# Séparation des caractéristiques (features) et de la cible

In [74]:
# Séparer les caractéristiques (features) et la cible

X = df.drop(columns=['sales_margin'])
bool_cols = X.select_dtypes(include='bool').columns
X[bool_cols] = X[bool_cols].astype(int)

y = df['sales_margin']

In [75]:
# Encoder les colonnes catégoriques avec LabelEncoder
label_encoders = {}
cat_columns = ['sales_agent', 'account', 'series', 'manager', 'office_location', 'regional_office', 'sector_entertainment', 'sector_finance', 
               'sector_marketing', 'sector_medical', 'sector_retail', 'sector_services', 'sector_software', 'sector_technolgy', 
               'sector_telecommunications', 'product_GTX Basic', 'product_GTX Plus Basic', 'product_GTX Plus Pro', 'product_GTX Pro', 
               'product_MG Advanced', 'product_MG Special']

for col in cat_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

KeyError: 'sector_entertainment'

# %%
# Division des données en ensembles d'entraînement et de test

In [ ]:

X = df.drop(columns=['sales_margin'])
y = df['sales_margin']


# Étape 1 : Remplacer les virgules par des points dans tout le dataframe
df = df.replace(',', '.', regex=True)

# Étape 2 : Convertir les colonnes en format numérique
for col in X.columns:  # Parcourt les colonnes sélectionnées dans X
    try:
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convertit au format numérique
    except Exception as e:
        print(f"Problème avec la colonne {col}: {e}")

        
# Normalisation des colonnes numériques
numerical_cols = ['sales_price', 'revenue','number_of_employees', 'engage_month', 'engage_day', 'engage_weekday']
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# %%
# Modélisation avec Gradient Boosting

In [ ]:

model = LGBMRegressor(random_state=42, n_estimators=500, learning_rate=0.05, max_depth=7)
model.fit(X_train, y_train)



LightGBMError: Feature (sector_entertainment) appears more than one time.


# %%
# Évaluation du modèle

In [ ]:

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.4f}")



# %%
# Visualisation de l'importance des caractéristiques

In [ ]:

importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
plt.barh(features, importances)
plt.xlabel("Importance")
plt.title("Importance des caractéristiques")
plt.show()



# %%
# Génération de données simulées pour prédiction

In [ ]:

future_data = pd.DataFrame({
    'sales_agent': [27, 29, 11, 15, 6],
    'account': [1, 2, 1, 2, 1],
    'series': [1, 1, 2, 3, 1],
    'manager': [3, 5, 1, 2, 4],
    'office_location': [1, 1, 1, 1, 1],
    'regional_office': [1, 2, 0, 2, 2],
    'year_founded_at': [1995, 2000, 1985, 2010, 2005],
    'revenue': [3178.24, 7708.38, 3178.24, 7708.38, 3178.24],
    'number_of_employees': [4540, 13756, 4540, 13756, 4540],
    'sales_cycle_days': [50, 60, 70, 80, 90],
    'sales_price': [1096.0, 1096.0, 55.0, 5482.0, 550.0],
    'sector_entertainment': [0, 0, 0, 0, 0],
    'sector_finance': [1, 1, 1, 1, 1],
    'sector_marketing': [0, 0, 0, 0, 0],
    'sector_medical': [0, 0, 0, 0, 0],
    'sector_retail': [0, 0, 0, 0, 0],
    'sector_services': [0, 0, 0, 0, 0],
    'sector_software': [0, 0, 0, 0, 0],
    'sector_technolgy': [0, 0, 0, 0, 0],
    'sector_telecommunications': [0, 0, 0, 0, 0],
    'product_GTX Basic': [1, 0, 0, 0, 0],
    'product_GTX Plus Basic': [0, 1, 0, 0, 0],
    'product_GTX Plus Pro': [0, 0, 1, 0, 0],
    'product_GTX Pro': [0, 0, 0, 1, 0],
    'product_MG Advanced': [0, 0, 0, 0, 1]
})
future_data_scaled = scaler.transform(future_data[numerical_cols])
future_predictions = model.predict(future_data_scaled)

# %%
# Visualisation des prédictions
start_index = 1300  # Début de l'affichage
plt.figure(figsize=(12, 6))

# Vraies valeurs
plt.plot(range(start_index, len(y_test)), y_test[start_index:], label="Vérités terrain", color="blue", linewidth=2)

# Prédictions actuelles
plt.plot(range(start_index, len(y_test)), y_pred[start_index:], label="Prédictions existantes", color="green", linestyle='-', linewidth=2)

# Prédictions futures
plt.plot(range(len(y_test), len(y_test) + len(future_predictions)), future_predictions, label="Prédictions futures", color="red", linestyle='--', linewidth=2)

plt.xlabel("Index temporel")
plt.ylabel("Sales Margin")
plt.title("Prédictions Gradient Boosting : Actuelles et Futures")
plt.legend()
plt.grid()
plt.show()